In [ ]:
%matplotlib
import sys
import os
from io import StringIO
sys.path.append('./DemoPD/Common')
sys.path.append('./TestData')
import tensorflow as tf
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import datetime 
import numpy as np
from getDataPD import runSQL_Insert, getDataPD, getMlData, runMlSQL_Insert
from sklearn.model_selection import train_test_split
import io
from PIL import Image

## 데이터불러오기 (Class 별 500)
- 업체로 부터 받은 corona, surface, void 데이터가 적어 전체 좌표에서 랜덤하게 x,y 좌표를 이동하며(coordiate translate) 인조데이터를 생성함
- 생성한 인조데이터에서 class 별 500 개씩 샘플링하여 학습데이터로 사용

In [ ]:
#void
void_sql = " SELECT * " +\
      " FROM pd_classf_train_data " +\
      " WHERE DATA_SERNO between 1 and 500"
void = getMlData(void_sql)

#surface
surface_sql = " SELECT * " +\
      " FROM pd_classf_train_data " +\
      " WHERE DATA_SERNO between 8344 and 8843"
surface = getMlData(surface_sql)

#coronaP
coronaP_sql = " SELECT * " +\
      " FROM pd_classf_train_data " +\
      " WHERE DATA_SERNO between 16687 and 17186"
coronaP = getMlData(coronaP_sql)

#coronaN
coronaN_sql = " SELECT * " +\
      " FROM pd_classf_train_data " +\
      " WHERE DATA_SERNO between 20531 and 21030"
coronaN = getMlData(coronaN_sql)

## 데이터 전처리
- csv 좌표데이터를 시각화 이미지로 만들기위해 데이터프레임으로 변환하여 PD 한 건 당 하나의 row로 저장

In [ ]:
#void preprocessing
void_list = []
for index, row in void.iterrows():
    void_list.append(row['PRPD_CONT'])
    
void_prpd=[]
columns = ['X', 'Y', 'CNT']
for i in range(len(void_list)): 
    prpd_ = pd.read_csv(StringIO(void_list[i]), delimiter = ',', encoding = 'utf-8', names = columns)
    void_prpd.append(prpd_)

In [ ]:
#surface preprocessing
surface_list = []
for index, row in surface.iterrows():
    surface_list.append(row['PRPD_CONT'])
    
surface_prpd=[]
columns = ['X', 'Y', 'CNT']
for i in range(len(surface_list)): 
    prpd_ = pd.read_csv(StringIO(surface_list[i]), delimiter = ',', encoding = 'utf-8', names = columns)
    surface_prpd.append(prpd_)

In [ ]:
#coronaP preprocessing
coronaP_list = []
for index, row in coronaP.iterrows():
    coronaP_list.append(row['PRPD_CONT'])
    
coronaP_prpd=[]
columns = ['X', 'Y', 'CNT']
for i in range(len(coronaP_list)): 
    prpd_ = pd.read_csv(StringIO(coronaP_list[i]), delimiter = ',', encoding = 'utf-8', names = columns)
    coronaP_prpd.append(prpd_)

In [ ]:
#coronaN preprocessing
coronaN_list = []
for index, row in coronaN.iterrows():
    coronaN_list.append(row['PRPD_CONT'])
    
coronaN_prpd=[]
columns = ['X', 'Y', 'CNT']
for i in range(len(coronaN_list)): 
    prpd_ = pd.read_csv(StringIO(coronaN_list[i]), delimiter = ',', encoding = 'utf-8', names = columns)
    coronaN_prpd.append(prpd_)

- 클래스별 데이터프레임을 이미지로 파일로 변환 후 다시 행렬로 변환(grey scale)

In [ ]:
data = []
for j in [void_prpd, surface_prpd, coronaP_prpd, coronaN_prpd]:
    for i in range(len(j)):
        # make an agg figure
        plt.figure(figsize=(3,2))
        plt.scatter(void_prpd[49]['X'], void_prpd[49]['Y'], s=0.05);
        plt.ylim(0,360)
        plt.xlim(0,360)
     
        buf = io.BytesIO()
        plt.savefig(buf, format='png')
        buf.seek(0)
        im =np.array(Image.open(buf).convert('L'))
        data.append(im)
        plt.clf();

- numpy 행렬 저장

In [ ]:
data_ = np.array(data)
data_.shape

#저장
np.save('dataset',data)

#로드
data = np.load('dataset.npy')
data.shape